Aggregate image level deep learning feature logistic model result to restaurants

In [1]:
import pandas as pd
import numpy as np

In [2]:
prefix=r'/Yelp/deep_logistic_result/'

In [26]:
# read prediction probability for all labels and images
numlabel=9

label=0
file=prefix+"prob_%d.csv" % label
df=pd.read_csv(file,header=None)
df.columns=['photo_id','%d' % label]

for label in range(1,numlabel):
    file=prefix+"prob_%d.csv" % label
    cache=pd.read_csv(file,header=None)
    cache.columns=['photo_id','%d' % label]
    df=df.merge(cache,on='photo_id')

In [27]:
df[:3]

,photo_id,0,1,2,3,4,5,6,7,8
0,306310,0.144380,0.738446,0.832474,0.350373,0.557770,0.881926,0.822641,0.443390,0.305312
1,414079,0.751620,0.207867,0.129508,0.457979,0.084926,0.335735,0.177415,0.177768,0.605128
2,235737,0.046159,0.928014,0.941867,0.350682,0.892192,0.961724,0.983470,0.752173,0.095722


In [24]:
file=r'/Yelp/test_photo_to_biz.csv'
p_biz_df = pd.read_csv(file)
p_biz_df[:3]

,photo_id,business_id
0,317818,003sg
1,30679,003sg
2,455084,003sg


In [28]:
# join image-restaurant map and prediction probabilities
df=df.merge(p_biz_df,on='photo_id')
df[:3]

,photo_id,0,1,2,3,4,5,6,7,8,business_id
0,306310,0.14438,0.738446,0.832474,0.350373,0.55777,0.881926,0.822641,0.44339,0.305312,06cko
1,306310,0.14438,0.738446,0.832474,0.350373,0.55777,0.881926,0.822641,0.44339,0.305312,424fd
2,306310,0.14438,0.738446,0.832474,0.350373,0.55777,0.881926,0.822641,0.44339,0.305312,cuof8


In [39]:
# average prediction probability for each restaurant
mean_prob=df.groupby(['business_id'])[np.arange(9).astype(np.str)].mean()
mean_prob[:3]

,0,1,2,3,4,5,6,7,8
business_id,,,,,,,,,
003sg,0.235450,0.598205,0.649850,0.512190,0.437117,0.725627,0.767255,0.388143,0.500284
00er5,0.232496,0.575878,0.625513,0.504930,0.387588,0.727191,0.766431,0.346793,0.543016
00kad,0.237391,0.557425,0.599383,0.493128,0.382931,0.706564,0.730700,0.353629,0.527460


In [40]:
# make prediction based on averaged probability
mean_pred = mean_prob.round()
mean_pred[:3]

,0,1,2,3,4,5,6,7,8
business_id,,,,,,,,,
003sg,0,1,1,1,0,1,1,0,1
00er5,0,1,1,1,0,1,1,0,1
00kad,0,1,1,0,0,1,1,0,1


In [41]:
mean_pred.to_csv(r'/Yelp/deep_logistic_result/mean_pred.csv',float_format='%d')

In [42]:
# take max prediction probability for each restaurant
max_prob=df.groupby(['business_id'])[np.arange(9).astype(np.str)].max()
max_prob[:3]

,0,1,2,3,4,5,6,7,8
business_id,,,,,,,,,
003sg,0.868324,0.995414,0.997553,0.886170,0.979756,0.999827,0.999522,0.919687,0.972309
00er5,0.772381,0.982432,0.991015,0.925846,0.961846,0.997650,0.998591,0.892391,0.987068
00kad,0.829464,0.967085,0.971796,0.897030,0.937014,0.994925,0.994631,0.862890,0.990153


In [43]:
max_pred = max_prob.round()
max_pred[:3]

,0,1,2,3,4,5,6,7,8
business_id,,,,,,,,,
003sg,1,1,1,1,1,1,1,1,1
00er5,1,1,1,1,1,1,1,1,1
00kad,1,1,1,1,1,1,1,1,1


In [44]:
max_pred.to_csv(r'/Yelp/deep_logistic_result/max_pred.csv',float_format='%d')

In [45]:
mix_pred = mean_pred.copy()

In [58]:
# use max probabilty for label 3 and 7 instead of average
mix_pred['3'] = max_pred['3']
mix_pred['7'] = max_pred['7']

In [47]:
mix_pred.to_csv(r'/Yelp/deep_logistic_result/mix_pred.csv',float_format='%d')

In [48]:
mix_pred[:3]

,0,1,2,3,4,5,6,7,8
business_id,,,,,,,,,
003sg,0,1,1,1,0,1,1,0,1
00er5,0,1,1,1,0,1,1,0,1
00kad,0,1,1,1,0,1,1,0,1


In [60]:
# sum(mix_pred['7']==1)
# sum(mean_pred['7']==1)

9760

In [50]:
with open(r'/Yelp/deep_logistic_result/mean_submission.csv','w') as fw:
    fw.write('business_id,labels\n')
    for ind in mean_pred.index:
        pred = list(mean_pred.loc[ind,:].astype(int))
        label = [str(i) for i in range(9) if pred[i]!=0]
        line = ind+','+' '.join(label)+'\n'
        fw.write(line)

In [59]:
with open(r'/Yelp/deep_logistic_result/mix_submission.csv','w') as fw:
    fw.write('business_id,labels\n')
    for ind in mix_pred.index:
        pred = list(mean_pred.loc[ind,:].astype(int))
        label = [str(i) for i in range(9) if pred[i]!=0]
        line = ind+','+' '.join(label)+'\n'
        fw.write(line)